The main goal is Predict the House Prices using Regression. 
Therefore, there would be finding the right features that helps the prediction less error.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## **1. Get The Data**

Take a look at the the Data Structure

In [ ]:
housing = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
housing.head(10)

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

### **Create a Test Set**

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

## **Discover and Visualize the Data to Gain Insights**


- In this phase, we will copying the training dataset to play with. 
- Check missing values data
- Looking for correlations
- Experimenting with Attribute Combinations

In [ ]:
housing1 = train_set.copy()

In [ ]:
housing1.isnull().sum().sort_values(ascending = False).head(30)

Features **PoolQC, MiscFeature, Alley,and Fence** have a lot of missing values.
We'll drop these features from the consideration

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(housing1.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset');

In [ ]:
housing1.drop(['PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1, inplace=True)

In [ ]:
housing1.head()

In [ ]:
corr = housing1.corr()
sns.heatmap(corr, linewidths=0.5);

In [ ]:
corr["SalePrice"].sort_values(ascending=False)

Checking Correlation from SalePrice attribute. 
There are several attributes has positive correlation with SalePrice which are: 
OverallQual, GrLivArea, GarageArea,TotalBsmtSF, 1stFlrSF, FullBath,TotRmsAbvGrd, YearBuilt, YearRemodAdd,GarageYrBlt, MasVnrArea

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["SalePrice", "OverallQual", "GrLivArea",
             'GarageArea','TotalBsmtSF', '1stFlrSF', 'FullBath',
              'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd','GarageYrBlt', 'MasVnrArea']
pd.plotting.scatter_matrix(housing1[attributes], figsize=(15, 10))


In [ ]:
attributes = ["SalePrice", "OverallQual", "GrLivArea",
             'GarageArea']
pd.plotting.scatter_matrix(housing1[attributes], figsize=(15, 10))

### Preparing the Data for Machine Learning Algorithms

In [ ]:
housing1 = train_set.drop("SalePrice", axis=1) # drop labels for training set
housing_labels = train_set["SalePrice"].copy()

In [ ]:
sample_incomplete_rows = housing1[housing1.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

Dropping the Object attributes to perform Imputer Median for missing numerical values

In [ ]:
housing_num = housing1.select_dtypes(exclude=['object'])
housing_num.head()

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

Transform the Training set:

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing1.index)

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

In [ ]:
imputer.strategy

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)
housing_tr.head()

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(housing_num.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset');

In [ ]:
plt.figure(figsize=(12,4))
sns.heatmap(housing_tr.isnull(),cbar=False,cmap='viridis',yticklabels=False)
plt.title('Missing value in the dataset');

In [ ]:
housing_cat = housing1.select_dtypes('object')
housing_cat.head(10)

Handling NaN Values 

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [ ]:
imp.fit(housing_cat)

In [ ]:
a = imp.transform(housing_cat)

In [ ]:
housing_object = pd.DataFrame(a, columns=housing_cat.columns,
                          index=housing1.index)

In [ ]:
housing_object.loc[sample_incomplete_rows.index.values]

In [ ]:
imp.strategy

In [ ]:
housing_object = pd.DataFrame(a, columns=housing_cat.columns,
                          index=housing_cat.index)
housing_object.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_object)
housing_cat_encoded[:5]

We have done replace the NaN in categorical values,

We have done shifting the object types into numbers categorical 

Now, we are missing the part combining all attributes into 1 place, or call Pipeline

 ### Transformation Pipelines

We will transform 2 pipelines here which are numerical pipeline and categorical pipeline. 
in the function, we have to create all transformation we have done along the way of creating the pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),

    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

In [ ]:
cat_pipeline = Pipeline([
        ('imp', SimpleImputer(strategy="most_frequent")),
        ('cat_encoder', OrdinalEncoder()),

    ])

housing_object_tr = cat_pipeline.fit_transform(housing_object)

In [ ]:
housing_object_tr

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = list(housing_object)

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing1)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

### Select and Train a Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = housing1.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

## Fine-Tune the Model

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:

from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

In [ ]:
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## Conclusion

The prediction could have done better by trying with some combinations with the attributes.

In [ ]:
final_model = grid_search.best_estimator_

X_test = test_set.drop("SalePrice", axis=1)
y_test = test_set["SalePrice"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)